In [20]:
import pandas as pd
import pyvisa
import time

resource_manager = pyvisa.ResourceManager()
Load=['GPIB0::5::INSTR']
Oscil=['USB0::0x0957::0x173D::MY50340340::0::INSTR']
Input=['USB0::0x0957::0xAC07::JPWC002359::0::INSTR']
DMM=['USB0::0x2A8D::0x1301::MY57224113::0::INSTR']
data=Load+Oscil+DMM+Input

DevicesDict={'Load':Load,'Oscilloscope':Oscil,'Input':Input,'Multimeter':DMM}
for item in data:
    if item in DevicesDict: 
        data[item]=DevicesDict[item]
print(DevicesDict)

list(DevicesDict.keys())
list(DevicesDict.values())

{'Load': ['GPIB0::5::INSTR'], 'Oscilloscope': ['USB0::0x0957::0x173D::MY50340340::0::INSTR'], 'Input': ['USB0::0x0957::0xAC07::JPWC002359::0::INSTR'], 'Multimeter': ['USB0::0x2A8D::0x1301::MY57224113::0::INSTR']}


[['GPIB0::5::INSTR'],
 ['USB0::0x0957::0x173D::MY50340340::0::INSTR'],
 ['USB0::0x0957::0xAC07::JPWC002359::0::INSTR'],
 ['USB0::0x2A8D::0x1301::MY57224113::0::INSTR']]

In [7]:
resource_manager.list_resources()

('GPIB0::5::INSTR',
 'USB0::0x0957::0x173D::MY50340340::0::INSTR',
 'USB0::0x0957::0xAC07::JPWC002359::0::INSTR',
 'USB0::0x2A8D::0x1301::MY57224092::0::INSTR',
 'USB0::0x2A8D::0x1301::MY57224113::0::INSTR',
 'USB0::0x2A8D::0x1301::MY57224258::0::INSTR',
 'USB0::0x2A8D::0x1301::MY57224696::0::INSTR',
 'USB0::0x2A8D::0x1401::MY57224258::0::INSTR')

In [2]:
df=pd.DataFrame.from_dict('Address','Device','Type')


ValueError: only recognize index or columns for orient

In [5]:
Load

['GPIB0::5::INSTR']

## AC Source Input

In [4]:


# You can change the variable name and resource name
Vin = resource_manager.open_resource(Input[0])
Vin.write("*RST") #Reset to default settings
Vin.write("OUTP:COUPLing DC")
Vin.write("SOUR:VOLT:OFFS 30")
Vin.write("SOUR:VOLTage:RANGE 270")
Vin.write("SOUR:CURRent:OFFSET 8")

Vin.write("OUTP 1") #Enable output
time.sleep(5)
voltage = Vin.query_ascii_values("MEAS:VOLT?")
current = Vin.query_ascii_values("MEAS:CURR?")
Vin.write("OUTP 0")
print("Voltage was "+str(voltage[0])+". Current was "+str(current[0])+".")


Voltage was 29.9498. Current was -0.0133115.


In [3]:
import pyvisa
import numpy as np

rm = pyvisa.ResourceManager()
instruments = np.array(rm.list_resources())

for instrument in instruments:
    my_instrument = rm.open_resource(instrument)
    try:
        identity = my_instrument.query('*IDN?')
        print("Resource: '" + instrument + "' is")
        print(identity + '\n')
    except pyvisa.VisaIOError:
        print('No connection to: ' + instrument)


Resource: 'GPIB0::5::INSTR' is
Agilent Technologies,N3300A,0,A.00.08


Resource: 'USB0::0x0957::0xAC07::JPWC002359::0::INSTR' is
Agilent,AC6803A,JPWC002359,A.01.02.0088




VisaIOError: VI_ERROR_RSRC_NFOUND (-1073807343): Insufficient location information or the requested device or resource is not present in the system.

## Load

In [2]:
Load=['GPIB0::5::INSTR']
resource_manager = pyvisa.ResourceManager()

OutLoad = resource_manager.open_resource(Load[0])

OutLoad.write("*RST")
OutLoad.write("INP ON")
OutLoad.write("CHANnel 2")
OutLoad.write("FUNC CURR")
OutLoad.write("CURR 1")
OutLoad.close()

## DMM

In [31]:
import pyvisa
import time
DMM=['USB0::0x2A8D::0x1301::MY57224113::0::INSTR']
resource_manager = pyvisa.ResourceManager()
VinDMM = resource_manager.open_resource(DMM[0])
VinDMM.write('DISP:TEXT "Iin Meas"')
VinDMM.write("*RST")
VinDMM.write("*CLS")
VinDMM.write("CONF:CURR:DC AUTO")
VinDMM.write("CURR:DC:TERMinals 10")
print(VinDMM.query("SYST:ERROR?"))
VinDMM.write("TRIG:DEL:AUTO ON")
VinDMM.query("*OPC?")
Vrcv=VinDMM.query("READ?")

# EQUIVALENT TO 'READ?': print(VinDMM.query("MEAS:VOLT:DC?"))
print(Vrcv)
print(VinDMM.query("SYST:ERROR?"))


#VinDMM.write("INIT")
#VinDMM.write("SENS: DATA2")
#VinDMM.write("")
#VinDMM.write("")
VinDMM.close()



+0,"No error"

+7.30833768E-01

+0,"No error"



## Oscilloscope

In [41]:
Oscil=['USB0::0x0957::0x173D::MY50340340::0::INSTR']
Measurement=['test1']
resource_manager = pyvisa.ResourceManager()
Oscil = resource_manager.open_resource(Oscil[0])
Oscil.encoding = 'ascii'


DMM_sequence=['Vinmeas','Iinmeas','Voutmeas','Ioutmeas','Pin','Pout','Eff']
Oscil_sequence=['Vinput','Vinput AC','Voutput','Voutput AC','Iload','Iload AC','Iinput','Iinput AC','Ioutput','Ioutput AC','Vinput sens','Iinput sens','Voutput sens','Ioutput sens','Iload sens']
# Test_sequence = DMM_sequence+Oscil_sequence
Test_sequence=['Vinput','Vinput AC','Voutput','Voutput AC','Iload','Iload AC','Iinput','Iinput AC','Ioutput','Ioutput AC','Vinput sens','Iinput sens','Voutput sens','Ioutput sens','Iload sens']
LoadCurr=['1','5'] # A
ConvMode=['Buck','Boost']
ConvModeVoltageDict={'Buck':'150','Boost':'80'}
files=['CSV', 'PNG']
# SAME AS THE OTHER ONE BUT CHATGPT WROTE IT ONE BY ONE. KEPT IN CASE MORE CHANGES NEEDE ETO BE MADE
# ChanConfLevel = {
#     'Vinput': ['50','500mV','2','2'],
#     'Vinput AC': ['100mV','500mV','2','2'],
#     'Voutput': ['50','500mV','2','2'],
#     'Voutput AC': ['100mV','500mV','2','2'],
#     'Iload': ['5','500mV','2','2'],
#     'Iload AC': ['5','500mV','2','500mA'],
#     'Iinput': ['5','500mV','2','2'],
#     'Iinput AC': ['5','500mV','2','500mA'],
#     'Ioutput': ['5','500mV','2','2'],
#     'Ioutput AC': ['5','500mV','2','500mA'],
#     'Vinput sens': ['5','500mV','2','2'],
#     'Iinput sens': ['5','500mV','2','2'],
#     'Voutput sens': ['5','500mV','2','2'],
#     'Ioutput sens': ['5','500mV','2','2'],
#     'Iload sens': ['50mV','500mV','2','2']
# }

ChanConfLevel = {}
for item in Test_sequence:
    if item in ['Iload', 'Iinput', 'Ioutput','Vinput sens','Iinput sens','Voutput sens','Ioutput sens','Iload sens']:
        ChanConfLevel[item] = ['5', '500mV', '2', '2']
    elif item in ['Vinput', 'Voutput']:
        ChanConfLevel[item] = ['50', '500mV', '2', '2']
    elif item in ['Vinput AC', 'Voutput AC']:
        ChanConfLevel[item] = ['100mV', '500mV', '2', '2']
    elif item in ['Iinput AC', 'Ioutput AC']:
        ChanConfLevel[item] = ['5', '500mV', '2', '500mA']
    elif item == 'Iload sens':
        ChanConfLevel[item] = ['50mV', '500mV', '2', '2']
    else:
        ChanConfLevel[item] = ['5', '500mV', '2', '2']

ACDCConf= {}
for item in Test_sequence:
    if item in ['Vinput AC', 'Voutput AC']:
        ACDCConf[item]=['AC','AC','DC','DC']
    elif item in ['Iinput AC', 'Ioutput AC','Iload AC']:
        ACDCConf[item]=['DC','DC','AC','AC']
    elif item in ['Iload sens']:
        ACDCConf[item]=['AC','DC','AC','AC']
    
    else:
        ACDCConf[item]=['DC','DC','DC','DC']

TrigConf={}
for item in Test_sequence:
    for val in LoadCurr:
        if item in ['Iload AC']:
            TrigConf[val,item]='0.5'
        else:
            TrigConf[val,item]=val



multiindex = pd.MultiIndex.from_product([ConvMode, LoadCurr], names=["ConvMode", "LoadCurr"])
df = pd.DataFrame(index=multiindex, columns=pd.MultiIndex.from_product([Oscil_sequence, files], names=["Measurement", "Files"]))

DMM_df=pd.DataFrame(index=multiindex, columns=pd.MultiIndex.from_product([DMM_sequence], names=["Measurement"]))

###############--CONFIGURE OSCILLOSCOPE--####################
Oscil.write('*RST')
Oscil.write("*CLS")
ChanNum=4
Oscil.write("TIMebase:MODE MAIN")
Oscil.write("TIMebase:POSition 0")
Oscil.write("TIMebase:RANGe 1 ms")
Oscil.write("TIMebase:REFerence CENTer")
Oscil.write("TRIGger:SWEep AUTO")
Oscil.write("TRIGger:NREJect 1")
Oscil.write("TRIGger:HFReject 1")
Oscil.write("TRIGger:MODE Edge")#Selects Trigger mode
Oscil.write("TRIGger:EDGE:SOURce CHAN3")#Trigger source is channel 3 
Oscil.write("TRIGger:EDGE:LEVel 3") #Trigger level at 2 A
for i in range(ChanNum):
    n=i+1
    Oscil.write(":CHAN{}:DISP ON".format(n))
    Oscil.write(":CHAN{}:BWLimit ON".format(n))
    Oscil.write(":CHAN3:SCALe 2") #config level
    Oscil.write(":CHAN3:OFFSet 3")
    # Oscil.write(":TRIG:EDGE:LEVEl {}".format(TrigConf['1','Iload AC']))#config trigger
    # Oscil.write(":CHAN{}:COUPling {}".format(n,ACDCConf['Iload'][i]))#config coupling


            # Oscil.write(":SAVE:FILEname '{}'".format(element))
            # print("Saving file {}".format(element))
            # NameSave = Oscil.query(":SAVE:FILEname?")
            # print('Saving file as {NameSave}'.format(NameSave))
            
            # Oscil.write(":SAVE:PWD '/usb0/prueba/'")
            # print(Oscil.query(":SAVE:PWD?"))
            
            # Oscil.write(":SAVE:IMAGe:FORMat PNG")
            # Oscil.write(":SAVE:IMAGe:INKSaver OFF")
            # Oscil.write(":SAVE:IMAGe:PALEtte COLOr")
            # Oscil.write(":SAVE:IMAGe:STARt {}".format(NameSave))
            # df.loc[(converter,load_current),(element,'PNG')]=NameSave+'.png'

            # Oscil.write(":SAVE:WAVeform:FORMat CSV")
            # Oscil.write(":SAVE:WAVeform:LENGth 1000")
            # Oscil.write(":SAVE:WAVeform:STARt {}".format(NameSave))
            # df.loc[(converter,load_current),(element,'CSV')]=NameSave+'.csv'




# for meas in Measurement:

#     Oscil.write('*RST')
#     Oscil.write("*CLS")
#     ChanNum=4
#     Oscil.write("TIMebase:MODE MAIN")
#     Oscil.write("TIMebase:POSition 0")
#     Oscil.write("TIMebase:RANGe 1 ms")
#     Oscil.write("TIMebase:REFerence CENTer")
#     Oscil.write("TRIGger:SWEep AUTO")
#     Oscil.write("TRIGger:NREJect 1")
#     Oscil.write("TRIGger:HFReject 1")
#     Oscil.write("TRIGger:MODE Edge")#Selects Trigger mode
#     Oscil.write("TRIGger:EDGE:SOURce CHAN3")#Trigger source is channel 3 
#     Oscil.write("TRIGger:EDGE:LEVel 2") #Trigger level at 2 A
#     CHsep=pd.DataFrame()
#     for i in range(ChanNum):
#         n=i+1
#         Oscil.write(":CHAN{}:DISP ON".format(n))
#         Oscil.write(":CHAN{}:BWLimit ON".format(n))
#         Oscil.write("WAVEForm:Source CHAN{}".format(n))
#         Oscil.write("WAVEForm:points max")
#         Oscil.write("WAVEForm:format ASCII")
#         c1=Oscil.query("WAVEform:DATA?")

#         c1=c1.replace('#800012999','')
#         c1=c1.split(',')
#         c1.append(meas for i in len(c1))

#         CHsep['CH{}'.format(n)]=[float(point) for point in c1]
#         # print('Esto es CH{}:   {}'.format(n,c1))
#     # create a list of tuples for each channel
        
#     # convert the list of tuples to a DataFrame
# # df = pd.DataFrame(CHsep, columns=['Measurement', 'Channel1', 'Channel2', 'Channel3', 'Channel4'])
#     # set the multi-index using the elements of Test_sequence
# # df = df.set_index('Measurement')
#     # Oscil.write(":SAVE:FILEname '{}'".format(meas))
#     # print(":SAVE:FILEname {}".format(meas))
#     # NameSave =Oscil.query(":SAVE:FILEname?")
#     # print(NameSave)
#     # Oscil.write(":SAVE:PWD '/usb0/prueba/'")
#     # print(Oscil.query(":SAVE:PWD?"))

#     # Oscil.write(":SAVE:IMAGe:FORMat PNG")
#     # Oscil.write(":SAVE:IMAGe:INKSaver OFF")
#     # Oscil.write(":SAVE:IMAGe:PALEtte COLOr")

#     # Oscil.write(":SAVE:IMAGe:STARt {}".format(NameSave))
#     # Oscil.write(":SAVE:WAVeform:FORMat CSV")
#     # Oscil.write(":SAVE:WAVeform:LENGth 1000")
#     # Oscil.write(":SAVE:WAVeform:STARt {}".format(NameSave))




df

Measurement       Vinput      Vinput AC      Voutput      Voutput AC       \
Files                CSV  PNG       CSV  PNG     CSV  PNG        CSV  PNG   
ConvMode LoadCurr                                                           
Buck     1           NaN  NaN       NaN  NaN     NaN  NaN        NaN  NaN   
         5           NaN  NaN       NaN  NaN     NaN  NaN        NaN  NaN   
Boost    1           NaN  NaN       NaN  NaN     NaN  NaN        NaN  NaN   
         5           NaN  NaN       NaN  NaN     NaN  NaN        NaN  NaN   

Measurement       Iload       ... Vinput sens      Iinput sens       \
Files               CSV  PNG  ...         CSV  PNG         CSV  PNG   
ConvMode LoadCurr             ...                                     
Buck     1          NaN  NaN  ...         NaN  NaN         NaN  NaN   
         5          NaN  NaN  ...         NaN  NaN         NaN  NaN   
Boost    1          NaN  NaN  ...         NaN  NaN         NaN  NaN   
         5          NaN  NaN  ...         NaN  NaN         NaN  NaN   

Measurement       Voutput sens      Ioutput sens      Iload sens       
Files                      CSV  PNG          CSV  PNG        CSV  PNG  
ConvMode LoadCurr                                                      
Buck     1                 NaN  NaN          NaN  NaN        NaN  NaN  
         5                 NaN  NaN          NaN  NaN        NaN  NaN  
Boost    1                 NaN  NaN          NaN  NaN        NaN  NaN  
         5                 NaN  NaN          NaN  NaN        NaN  NaN  

[4 rows x 30 columns]

In [30]:
Oscil.query(":SAVE:FILEname?")
bool(Oscil.query("*opc?"))

True